# Объединение датафреймов

# Домашнее задание

Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [31]:
import numpy as np
import pandas as pd
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
#ratings['timestamp'] = pd.Timedelta(ratings['timestamp'])
joined = ratings.merge(movies, on='movieId', how='left')
count_ratings= joined.groupby(by='userId')['rating'].count()
count_ratings = count_ratings[count_ratings > 100]
joined = joined[joined['userId'].isin(count_ratings.index)]
life_time = joined.groupby(by='userId')['timestamp'].max() - joined.groupby(by='userId')['timestamp'].min()
pd.Timedelta(life_time.mean())

Timedelta('0 days 00:00:00.040080')

Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [32]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [33]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [34]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [35]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [45]:
table_1 = client_base[['client_id']].merge(rzd.astype('Int64'), on='client_id', how='left')\
.merge(auto.astype('Int64') , on='client_id', how='left').merge(air.astype('Int64') , on='client_id', how='left').fillna(0)
table_2 = client_base.merge(rzd.astype('Int64'), on='client_id', how='left')\
.merge(auto.astype('Int64') , on='client_id', how='left').merge(air.astype('Int64') , on='client_id', how='left').fillna(0)

table_2

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093,0,0
1,112,Энтузиастов 8а,2810,0,0
2,113,Левобережная 1а,10283,57483,0
3,114,Мира 14,5774,83,0
4,115,ЗЖБИиДК 1,981,912,81
5,116,Строителей 18,0,4834,4
6,117,Панфиловская 33,0,98,13
7,118,Мастеркова 4,0,0,173


# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1. Широта и долгота дают понимание о примерном расположнии точки продаж, но есть исключения, если это, например, одно здание и 2 точки продаж на разных этажах. 
2. Прежде всего надо объединить все таблицы аутер джойном,  чтобы не терять доп данные, по user_ID. 
3. Если при этом в таблице визитов и покупок у нас будет известен адрес покупателя, и точки расположения магазинов, мы будем понимать, в какой ему заходить удобнее. Либо если мы знаем хотя бы адрес магазина, поймем, что он этот покупатель скорее всего живет или работает где-то рядом (в зависимости от характера и частоты покупок).
4. Также раз нам известно, когда требовалась широта и долгота, можно понимать timestamp и его добавить в таблицу - если это выходные, скорее всего человеку удобнее туда заходить за покупками из дома,  если будни среди дня - около работы.